In [14]:
setwd("C:/Users/xiaoq/PycharmProjects/Algorithm Bioinfo")

In [ ]:
## download the data
download.file("http://kaell.org/files/survivalLIHC.txt", destfile = "SurvivalData.txt")

In [ ]:
SurvivalData = read.delim("SurvivalData.txt", header = TRUE)

In [ ]:
## for later time and event addition in Excel
write.csv(SurvivalData, "SurvivalData_Overall.csv")

In [ ]:
SurvivalData = SurvivalData[-c(1:6),]
row.names(SurvivalData) = SurvivalData$TCGA.IDs
SurvivalData = SurvivalData[,-1]

In [ ]:
## Since the R didnt recognise the numeric value in the txt file, I save it in csv and then open it, and also for late transpose.
write.csv(SurvivalData, "SurvivalData.csv")
SurvivalData = read.csv("SurvivalData.csv", header = TRUE)

In [ ]:
## When Reading the file, gene names in the first column was interpret as x, which are the genes. 
row.names(SurvivalData) = SurvivalData$X
SurvivalData = SurvivalData[,-1]
View(SurvivalData)

In [ ]:
## The genes(rows) containing value/values of 0 will be removed(They are not intesting).
Expression_filter = apply(SurvivalData, 1, function(x) sum(x ==0) ==0)
SurvivalData = SurvivalData[Expression_filter,]

In [ ]:
## Filter the genes with low levels(not so intesting)
Mean_filter = apply(SurvivalData, 1, function(x) mean(x) > 5)
SurvivalData = SurvivalData[Mean_filter,]
write.csv(SurvivalData, "SurvivalData_Filtered.csv")

In [ ]:
##Combine the "Event" and "Time" information in Excel after the step above
SurvivalData = read.csv("SurvivalData_Filtered.csv")

In [ ]:
## Transpose the data to the format for survival analysis(colanmes = genename, and rownames = patientID)
SurvivalData = t(SurvivalData)
write.csv(SurvivalData, "SurvivalData_Filtered_t.csv")

In [ ]:
## After the above step, Delete the first row(nonsense when saved as csv file by R) in Excel
SurvivalData = read.csv("SurvivalData_Filtered_t.csv", header = TRUE)

In [38]:
#########################Run the survival analysis(log-rank test)
library(tidyverse)
library(survival)
library(emil)

In [39]:
## Define the expression level according to median
for (i in colnames(SurvivalData)[c(4:length(SurvivalData))]){
  SurvivalData[,i] = ifelse(SurvivalData[,i] > median(SurvivalData[,i]), "high", "low")
}

In [40]:
## Define the event and time
SurvivalData = SurvivalData %>% mutate(Event = ifelse(Status == "alive", 1, 0))
time <- SurvivalData$LivingDays
event <- SurvivalData$Event

In [41]:
## log_rank test with original P values
chiscores = apply(SurvivalData[c(4:5584)], 2, function(x) survdiff(Surv(time, event) ~ x, data = SurvivalData[c(4:5584)], rho = 0) )
P_valueCalculation = lapply(chiscores, function(x) pvalue(x, log_p = FALSE, n = length(chiscores)))
P_values = do.call(rbind, P_valueCalculation)
write.csv(P_values, "all_pvalue.csv")
P_values = read.csv("all_pvalue.csv",sep = ",")
colnames(P_values)[1] = "Genename"
chiscores = do.call(rbind, chiscores)
chiscores = chiscores[,c(1,5)]
chiscores = as.data.frame(chiscores)
chiscores$Genename = rownames(chiscores)
chiscores = merge(chiscores, P_values, by = "Genename", all.x = TRUE)
colnames(chiscores)[4] = "P_Value"

ERROR: Error in survdiff.fit(y, groups, strata.keep, rho): There is only 1 group


In [42]:
## Correct the P value by FDR in the dataframe
P_val_vec = chiscores[[4]]
Pval_FDR = p.adjust(P_val_vec, method = "fdr")
chiscores = cbind(chiscores, Pval_FDR)
View(chiscores)

ERROR: Error in View(chiscores): 'View()' not yet supported in the Jupyter R kernel


In [43]:
##Count the numbers with significant P_values and corrected P values
NumberofSigP = nrow(chiscores[which(chiscores$P_Value < 0.05),])
NumberofSigCorrectP = nrow(chiscores[which(chiscores$Pval_FDR < 0.05),])
NumberofSigP
NumberofSigCorrectP

[1] 203

[1] 0